# UHS4 imputation
__Author__: Jesse Marks <br>

**GitHub Issue:** [Issue #117](https://github.com/RTIInternational/bioinformatics/issues/117)

This document logs the steps taken to perform pre-imputation procedures on the UHS4 dataset, both EA and AA. The starting point for this analysis is after quality control of observed genotypes. The quality controlled genotypes are oriented on the GRCh37 plus strand. 

## Software and tools
The software and tools used for porcessing these data are
* [Michigan Imputation Server](https://imputationserver.sph.umich.edu/index.html) (MIS)
* [Amazon Web Services (AWS) - Cloud Computing Services](https://aws.amazon.com/)
    * Linux AMI
* [PLINK v1.90 beta 4.10](https://www.cog-genomics.org/plink/)
* [bgzip](http://www.htslib.org/doc/tabix.html)
* [BCF Tools](http://www.htslib.org/doc/bcftools.html)
* Windows 10 with [Cygwin](https://cygwin.com/) installed
* GNU bash version 4.2.46

## Data retrieval and organization
PLINK binary filesets will be obtained from AWS S3 storage.

Jesse Marks performed the QC and stored the observed genotype data at:

```
s3/rti-heroin/uhs4/data/genotype/observed/final/
```

###  Statistics Breakdown 
This table includes the initial number of variants in each study as well as the final number of variants in the intersection set. The `Variants Post-Filtering` is in referral to the filtering steps (1) remove discordant alleles & (2) removal of monomorphic variants.

#### EA
| Data Set      | Initial Variants | Variants Post-Filtering  | Intersection     |
|---------------|------------------|--------------------------|------------------|
| UHS4          |                  |                          | NA               |

#### AA
| Data Set      | Initial Variants | Variants Post-Filtering  | Intersection     |
|---------------|------------------|--------------------------|------------------|
| UHS4          |                  |                          | NA               |

## Create Directory Structure & Download Data
The following section needs to be modified each time to reflect where the data is stored!

In [1]:
### EC2 command line (bash) ###

# Modify variables below
######################################################################
base_dir=/shared/jmarks/heroin/uhs4/genotype/imputed/004
genD=/shared/jmarks/heroin/uhs4/genotype/observed/final/004

base_name="chr_all" # chr23 or chr_all
chr_list={1..23} # or {1..22} 
ancestry_list="ea aa" # space delimited Ex. "ea aa ha"
study_list="uhs4" # space delimited, lowercase
######################################################################

mkdir ${base_dir}/processing/{intersect,1000g,impute_prep}
for study in ${study_list};do
    for ancestry in ${ancestry_list};do
        mkdir -p ${base_dir}/processing/${study}
        mkdir -p ${base_dir}/data/${study}/genotype/observed/${ancestry}
    done
done

## download genotype (with AWS CLI tools) to respective directories ##
#aws s3 sync s3://rti-heroin/kreek/data/genotype/original/20181128/ \
#    ${base_dir}/data/${study}/genotype/observed/
#mv ${base_dir}/data/${study}/genotype/observed/ea* ${base_dir}/data/${study}/genotype/observed/ea
#mv ${base_dir}/data/${study}/genotype/observed/aa* ${base_dir}/data/${study}/genotype/observed/aa

# Data Processing
## GRCh37 strand and allele discordance check
### MAF for study data

In [ ]:
### EC1 command line (Bash) ###

# write out the MAF report
for study in ${study_list}; do
    study_dir=${base_dir}/processing/${study}/strand_check
    mkdir ${study_dir}
    for ancestry in ${ancestry_list};do
        data_dir=${base_dir}/data/${study}/genotype/observed/${ancestry}
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bed ${data_dir}/*bed\
            --bim ${data_dir}/*bim\
            --fam ${data_dir}/*fam\
            --freq \
            --out ${study_dir}/${ancestry}_${base_name}
    done
done

# Get list of variants from all studies
for ancestry in ${ancestry_list}; do
    for study in ${study_list};do
        cat ${base_dir}/data/${study}/genotype/observed/${ancestry}/*bim | \
                perl -lane 'if (($F[0]+0) <= 23) { print $F[1]; }' | \
                sort -u > ${base_dir}/processing/${ancestry}_${base_name}_sorted_variants.txt
    done
     wc -l ${base_dir}/processing/$ancestry*
done

for ancestry in ${ancestry_list}; do

```
2006753 /shared/jmarks/heroin/uhs4/genotype/imputed/004/processing/ea_chr_all_sorted_variants.txt
1627424 /shared/jmarks/heroin/uhs4/genotype/imputed/004/processing/aa_chr_all_sorted_variants.txt
 ```

### MAF for 1000G
This pipeline is currently set up to handle EUR and AFR populations.
#### Autosomes
Get 1000G MAF for chromosomes 1–22 (autosomes).

In [ ]:
### EC2 command line (Bash)

# Calculate autosome MAFs for 1000G populations
for ancestry in ${ancestry_list}; do

    if [ $ancestry == "ea" ]; then
        pop="EUR"
    elif [ $ancestry == "ha" ]; then
        pop="AMR";
    else
        pop="AFR"
    fi
    
    for chr in {1..22}; do
        /shared/bioinformatics/software/scripts/qsub_job.sh \
            --job_name ${pop}_${chr}_MAF \
            --script_prefix ${base_dir}/processing/1000g/${pop}_chr${chr}.maf \
            --mem 6.8 \
            --nslots 2 \
            --priority 0 \
            --program /shared/bioinformatics/software/perl/stats/calculate_maf_from_impute2_hap_file.pl \
                --hap /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chr${chr}.hap.gz\
                --legend /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chr${chr}.legend.gz \
                --sample /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3.sample \
                --chr ${chr} \
                --out ${base_dir}/processing/1000g/${pop}_chr${chr}.maf \
                --extract ${base_dir}/processing/${ancestry}_${base_name}_sorted_variants.txt \
                --keep_groups ${pop}
    done
done

#### chrX
Get 1000G MAF for chromosome 23 (chrX).

In [ ]:
chr=23
for ancestry in ${ancestry_list};do

    if [ $ancestry == "ea" ]
    then
        pop="EUR"
    elif [ $ancestry == "ha" ]; then
        pop="AMR"
    else
        pop="AFR"
    fi

    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${pop}_23_MAF \
        --script_prefix ${base_dir}/processing/1000g/${pop}_chr${chr}.maf \
        --mem 6.8 \
        --nslots 2 \
        --priority 0 \
        --program /shared/bioinformatics/software/perl/stats/calculate_maf_from_impute2_hap_file.pl \
            --hap /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chrX_NONPAR.hap.gz\
            --legend /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chrX_NONPAR.legend.gz \
            --sample /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3.sample \
            --chr $chr \
            --out ${base_dir}/processing/1000g/${pop}_chr${chr}.maf \
            --extract ${base_dir}/processing/${ancestry}_${base_name}_sorted_variants.txt \
            --keep_groups ${pop}
done

### Merge 1000G chromosomes
Only need to perform this if there were multiple chromosomes for which the MAF was calculated—e.g. more than just chrX.

In [ ]:
### Bash ###

# Merge per chr MAFs for each 1000G population
for ancestry in ${ancestry_list};do
    if [ $ancestry == "ea" ]
    then
        pop="EUR"
    elif [ $ancestry == "ha" ]; then
        pop="AMR"
    else
        pop="AFR"
    fi
    
    head -n 1 ${base_dir}/processing/1000g/${pop}_chr1.maf > ${base_dir}/processing/1000g/${pop}_chr_all.maf
    tail -q -n +2 ${base_dir}/processing/1000g/${pop}_chr??maf  >> ${base_dir}/processing/1000g/${pop}_chr_all.maf
    
    wc -l $base_dir/processing/1000g/${pop}_chr_all.maf
done

```
1069721 /shared/jmarks/heroin/uhs4/genotype/imputed/004/processing/1000g/EUR_chr_all.maf
899125 /shared/jmarks/heroin/uhs4/genotype/imputed/004/processing/1000g/AFR_chr_all.maf
```

###  Allele Discordances Check
The allele discordances will be resolved by
* Flipping allele discordances that are fixed by flipping
* Removing SNPs with discordant names
* Removing SNPs with discordant positions
* Removing allele discordances that are not resolved by flipping
* Removing alleles with large deviations from the reference population allele frequencies

Given that the allele discordance check was done using a union set of SNPs across all studies within an ancestry group, some of the SNPs logged as discordant for a given study may not actually be in the study. Fortunately, if they are not in a given study they will not interfere with the filtering procedures. Note that the intersection set is used for the final studies merger.

#### Autosomes

In [ ]:
### Bash ###

# Run discordance checks for each ancestry group
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        if [ $ancestry = "ea" ]; then
            pop=EUR
        elif [ $ancestry = "ha" ]; then
            pop=AMR
        else
            pop=AFR
        fi

       /shared/bioinformatics/software/scripts/qsub_job.sh \
           --job_name ${ancestry}_${study}_crosscheck \
           --script_prefix ${base_dir}/processing/$study/strand_check/${ancestry}_allele_discordance_check \
           --mem 6 \
           --nslots 4 \
           --priority 0 \
           --program "Rscript /shared/bioinformatics/software/R/check_study_data_against_1000G.R
               --study_bim_file ${base_dir}/data/${study}/genotype/observed/${ancestry}/*bim
               --study_frq_file ${base_dir}/processing/${study}/strand_check/${ancestry}_chr_all.frq
               --ref_maf_file ${base_dir}/processing/1000g/${pop}_chr_all.maf
               --out_prefix ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance"
    done
done

#### chrX 
Not necessary to run unless you are only processing chrX.

In [ ]:
#for study in ${study_list}; do
#    for ancestry in ${ancestry_list};do
#        if [ $ancestry = "ea" ]; then
#            pop=EUR
#        elif [ $ancestry == "ha" ]; then
#            pop=AMR
#        else
#            pop=AFR
#        fi
#
#        # chr23 discordance check
#        /shared/bioinformatics/software/scripts/qsub_job.sh \
#            --job_name ${ancestry}_${study}_crosscheck_chr$chr \
#            --script_prefix ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance_check \
#            --mem 6.8 \
#            --nslots 1 \
#            --priority 0 \
#            --program "Rscript /shared/bioinformatics/software/R/check_study_data_against_1000G.R
#                --study_bim_file ${base_dir}/data/${study}/genotype/observed/${ancestry}/*bim
#                --study_frq_file ${base_dir}/processing/${study}/strand_check/${ancestry}_chr23.frq
#                --ref_maf_file ${base_dir}/processing/1000g/${pop}_chr23.maf
#                --out_prefix ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance"
#    done
#done

### Resolving Allele Discordances

In [ ]:
# Apply filters
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        echo -e "\n===============\nProcessing ${study}_${ancestry}\n"
        echo "Making remove list"
        cat <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_not_fixed_by_strand_flip | tail -n +2) \
            <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.at_cg_snps_freq_diff_gt_0.2 | tail -n +2) \
            <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_names | tail -n +2) \
            <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_positions | tail -n +2) \
            <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_polymorphic_in_study_not_fixed_by_strand_flip | tail -n +2) | \
              sort -u > ${base_dir}/processing/${study}/strand_check/${ancestry}_snps.remove

        # Create flip list
        echo "Making flip list"
        comm -23 <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles | tail -n +2 | sort -u) \
                 <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_not_fixed_by_strand_flip | tail -n +2 | sort -u) \
                 > ${base_dir}/processing/${study}/strand_check/${ancestry}_snps.flip

        # Apply filters
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bed     ${base_dir}/data/${study}/genotype/observed/${ancestry}/*bed \
            --bim     ${base_dir}/data/${study}/genotype/observed/${ancestry}/*bim \
            --fam     ${base_dir}/data/${study}/genotype/observed/${ancestry}/*fam \
            --exclude ${base_dir}/processing/${study}/strand_check/${ancestry}_snps.remove \
            --flip    ${base_dir}/processing/${study}/strand_check/${ancestry}_snps.flip \
            --make-bed \
            --out     ${base_dir}/processing/${study}/${ancestry}_filtered
    done
done

wc -l $base_dir/processing/*/*bim

 ```
  1627393 /shared/jmarks/heroin/uhs4/genotype/imputed/004/processing/uhs4/aa_filtered.bim
  2006720 /shared/jmarks/heroin/uhs4/genotype/imputed/004/processing/uhs4/ea_filtered.bim
```

## Remove monomorphic variants
Monomorphic variants prevent MIS from accepting the genotype data. In this case, an arbitrarily small MAF is set that is smaller than the lower bound for these data.

In [ ]:
### Bash ###

# Apply filters
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        geno_dir=${base_dir}/processing/${study}

        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bfile ${geno_dir}/${ancestry}_filtered \
            --maf 0.000001 \
            --make-bed \
            --out ${geno_dir}/${ancestry}_filtered_mono
    done
done

wc -l $base_dir/processing/*/*mono.bim

```
  1590276 /shared/jmarks/heroin/uhs4/genotype/imputed/004/processing/uhs4/aa_filtered_mono.bim
  1792893 /shared/jmarks/heroin/uhs4/genotype/imputed/004/processing/uhs4/ea_filtered_mono.bim
```

### SNP intersection
Only run if merging multiple data sets.

In [ ]:
#studies=($study_list)  #studies=(UHS1 UHS2 UHS3_v1-2 UHS3_v1-3) # array of study names
#num=${#studies[@]}
#
## Get intersection set
#for ancestry in ${ancestry_list};do
#    bim_files=()
#    for (( i=0; i<${num}; i++ ));do
#        bim_files+=(${base_dir}/processing/${studies[$i]}/${ancestry}_filtered_mono.bim)
#    done
#    
#    echo -e "\nCalculating intersection between $ancestry ${study_list}...\n"
#    sort ${bim_files[@]} | uniq -dc | awk -v num=$num '$1 == num {print $3}' \
#        > ${base_dir}/processing/intersect/${ancestry}_variant_intersection.txt
#
#    # Make new PLINK binary file sets
#    for study in ${studies[@]}; do
#        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#            --noweb \
#            --bfile ${base_dir}/processing/${study}/${ancestry}_filtered_mono \
#            --extract ${base_dir}/processing/intersect/${ancestry}_variant_intersection.txt \
#            --make-bed \
#            --out ${base_dir}/processing/intersect/${study}_${ancestry}_filtered_snp_intersection
#    done
#done

```wc -l *txt

```

### Merge test
If merging multiple datasets together:

As a final check to confirm that our data sets are all compatible, a PLINK file set merge is conducted. If any issues persist then an error will be raised.

In [ ]:
#for ancestry in $ancestry_list;do
#
#    echo "Creating $ancestry merge-list"
#    touch ${base_dir}/processing/intersect/${ancestry}_merge_list.txt
#    for study in $study_list;do
#        echo ${base_dir}/processing/intersect/${study}_${ancestry}_filtered_snp_intersection >>\
#             ${base_dir}/processing/intersect/${ancestry}_merge_list.txt
#    done
#
## Merge file sets
#    echo -e "\n\n======== ${ancestry} ========\n\n"
#    /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#        --noweb \
#        --memory 4000 \
#        --merge-list ${base_dir}/processing/intersect/${ancestry}_merge_list.txt \
#        --make-bed \
#        --out ${base_dir}/processing/intersect/${ancestry}_studies_merged
#done

```
 wc -l *merged.bim
```

## Imputation preparation for Michigan Imputation Server
Visit the [MIS Getting Started Webpage](https://imputationserver.sph.umich.edu/start.html#!pages/help) for more information about the preparing the data for upload to MIS.


### VCF File Conversion

In [ ]:
### Split by chr and remove any individuals with missing data for whole chr

## if merged data sets together
#for ancestry in $ancestry_list;do
#    # Remove SNPs
#    /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#        --noweb \
#        --memory 4000 \
#        --bfile ${base_dir}/processing/intersect/${ancestry}_studies_merged \
#        --chr ${chr} \
#        --mind 0.99 \
#        --make-bed \
#        --out ${base_dir}/processing/impute_prep/${ancestry}_chr${chr}_for_phasing 
#done > ${base_dir}/processing/impute_prep/chr_splitting.log 

## if NO merging was done
for ancestry in $ancestry_list;do
    for chr in {1..23};do
        # Remove SNPs
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 4000 \
            --bfile ${base_dir}/processing/$study/${ancestry}_filtered_mono \
            --chr ${chr} \
            --mind 0.99 \
            --make-bed \
            --out ${base_dir}/processing/impute_prep/${ancestry}_chr${chr}_for_phasing 
    done
done > ${base_dir}/processing/impute_prep/chr_splitting.log 

for ancestry in $ancestry_list; do
    grep removed $base_dir/processing/impute_prep/$ancestry*log |
        perl -lne '/(\d+)(\speople)/;
             $mycount += $1; 
             print $mycount if eof'  > $base_dir/processing/impute_prep/$ancestry.removed
    any_removed=$(cat $base_dir/processing/impute_prep/$ancestry.removed)
    if [ "$any_removed" == 0 ]; then
        echo "No $ancestry subjects removed"
    else
        echo "Some $ancestry subjects removed"
    fi
done

__Note__: No subjects were removed.

In [ ]:
# EC2 command line #

for chr in {1..23};do
    for ancestry in ${ancestry_list};do
        final_dir=${base_dir}/processing/impute_prep/${ancestry}
        mkdir $final_dir
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 5000 \
            --bfile ${final_dir}/../${ancestry}_chr${chr}_for_phasing \
            --output-chr M \
            --set-hh-missing \
            --recode vcf bgz \
            --out ${final_dir}/${ancestry}_chr${chr}_final
    done
done

Transfer the *.vcf.gz files to local machine (per chromosome) and then upload to MIS.

## Upload to Michigan Imputation Server (MIS)

### Uploading parameters
These are the parameters that were selected on MIS.

__Name__: UHS4_HA

__Reference Panel__ 1000G Phase 3 v5

__Input Files__ File Upload <br>

* Select Files - select VCF (.gz) files that were downloaded to local machine from cloud. <br>

__Phasing__: ShapeIT v2.r790 (unphased) 

__Population__: AMR

__Mode__: Quality Control & Imputation

* I will not attempt to re-identify or contact research participants.
* I will report any inadvertent data release, security breach or other data management incident of which I become aware.

# Download Imputed Results from MIS
First, download the data form the Michigan Imputation Server (MIS) by clicking on the link provided in the email they send out to alert you that your data has finished. Here you will find commands for downloading the data.

## AA
The zip files from Michigan Imputation Server (MIS) need to be inflated before you can begin working with them. They require a passcode that is sent by MIS to email.
* password: vMA#GHW3ddj5cf

Then, inflate imputation results.

In [ ]:
ancestry=aa
passW="'vMA#GHW3ddj5cf'"
cd /shared/jmarks/heroin/uhs4/genotype/imputed/004/final/$ancestry

## dowload_data_$ancestry
#================================================================================ 
# QC-results
wget https://imputationserver.sph.umich.edu/share/results/87dfa732a018a5c43d93bd33b8c66fe2/qcreport.html

# SNP Statistics
wget https://imputationserver.sph.umich.edu/share/results/b7626e2d062d088b27934aa16c7ec36b/statistics.txt

# Logs
wget https://imputationserver.sph.umich.edu/share/results/b9d9d24d235ed5d3eebdb5fe2b29f411/chr_1.log
wget https://imputationserver.sph.umich.edu/share/results/2e6bb7fd10abe3cf39ae7c80291497c7/chr_10.log
wget https://imputationserver.sph.umich.edu/share/results/47b9144a20d6b5bb902a088aef186f36/chr_11.log
wget https://imputationserver.sph.umich.edu/share/results/165097a70ad98d434ee44df1b4193542/chr_12.log
wget https://imputationserver.sph.umich.edu/share/results/b1d022d0747b5ccb212f86c2b1955a3b/chr_13.log
wget https://imputationserver.sph.umich.edu/share/results/96b78b18dcf9a33607d96dac1d859552/chr_14.log
wget https://imputationserver.sph.umich.edu/share/results/f3c3e0c89acf799d53473c2f8be1698f/chr_15.log
wget https://imputationserver.sph.umich.edu/share/results/86ffc92788410e68953f6f16240fb984/chr_16.log
wget https://imputationserver.sph.umich.edu/share/results/5df3c02b53fccb37968e6d699ea7f249/chr_17.log
wget https://imputationserver.sph.umich.edu/share/results/a310ccd27cc24193eff504a01f554af7/chr_18.log
wget https://imputationserver.sph.umich.edu/share/results/b448faea06fb9f505f4abd37eab8e97a/chr_19.log
wget https://imputationserver.sph.umich.edu/share/results/dd820dbffaf978df5da695ddaf359522/chr_2.log
wget https://imputationserver.sph.umich.edu/share/results/658ac06db7c8e7adcfdf467771f8fcb4/chr_20.log
wget https://imputationserver.sph.umich.edu/share/results/78a734d302d584bc2c1a0e5405928de8/chr_21.log
wget https://imputationserver.sph.umich.edu/share/results/7e05e40c1089eb7c81489e27139f890b/chr_22.log
wget https://imputationserver.sph.umich.edu/share/results/49a746ad4207b10d494efce17a6c87f/chr_3.log
wget https://imputationserver.sph.umich.edu/share/results/f94a487ad3d1a7749c1c1fdb0db493eb/chr_4.log
wget https://imputationserver.sph.umich.edu/share/results/b98046c8e6d180cbc81a21d0d236a9f6/chr_5.log
wget https://imputationserver.sph.umich.edu/share/results/72c7d22baa2f093479cc5a7de92c6b5/chr_6.log
wget https://imputationserver.sph.umich.edu/share/results/1272b00560e2b74d8212e68701032c80/chr_7.log
wget https://imputationserver.sph.umich.edu/share/results/d50e8cbde9117cba7dd15d8699e59d88/chr_8.log
wget https://imputationserver.sph.umich.edu/share/results/8463731695ffe32795b0d07a5f8123eb/chr_9.log

# Imputation Results
wget https://imputationserver.sph.umich.edu/share/results/cd86f2114d5ed93b1ae82529b791d27e/chr_1.zip
wget https://imputationserver.sph.umich.edu/share/results/84b90f47066e7220e141b8b3b6c21db0/chr_10.zip
wget https://imputationserver.sph.umich.edu/share/results/48c64608d3c3b23e5cefe8675b102684/chr_11.zip
wget https://imputationserver.sph.umich.edu/share/results/a8c95c04496ba390120f6a57b4f9ea62/chr_12.zip
wget https://imputationserver.sph.umich.edu/share/results/1c244be4219e2e1081232c72909b2852/chr_13.zip
wget https://imputationserver.sph.umich.edu/share/results/1a151988b48d0b99953322cf28e4161f/chr_14.zip
wget https://imputationserver.sph.umich.edu/share/results/40c5bb037a8b9702346bdf3228a43063/chr_15.zip
wget https://imputationserver.sph.umich.edu/share/results/3186f340643902a2106c4f774b12f267/chr_16.zip
wget https://imputationserver.sph.umich.edu/share/results/f141baf700d6bc0ffac05d8e796fc6b/chr_17.zip
wget https://imputationserver.sph.umich.edu/share/results/299e3317f4d3e05a8b0adebfde555f73/chr_18.zip
wget https://imputationserver.sph.umich.edu/share/results/85d661c426704ab58a8ed2dff86dbbb3/chr_19.zip
wget https://imputationserver.sph.umich.edu/share/results/3e8ca03982a1f141f3f840ef4e7cb37/chr_2.zip
wget https://imputationserver.sph.umich.edu/share/results/b4aa231f198e4448263de1dd6e300238/chr_20.zip
wget https://imputationserver.sph.umich.edu/share/results/c3d7eb327f2c915b002dc800ec190f6e/chr_21.zip
wget https://imputationserver.sph.umich.edu/share/results/77d843226b9313525921a1cb6b16dcff/chr_22.zip
wget https://imputationserver.sph.umich.edu/share/results/2b18e6986c0908c1e77842efa811abce/chr_3.zip
wget https://imputationserver.sph.umich.edu/share/results/9418ed8c36a6af7157350b100af5dab2/chr_4.zip
wget https://imputationserver.sph.umich.edu/share/results/43e47c9b0a9aae457548207a211a18fd/chr_5.zip
wget https://imputationserver.sph.umich.edu/share/results/926264c9082b85a402a884c415b1a351/chr_6.zip
wget https://imputationserver.sph.umich.edu/share/results/cc1108e54855b24a7d2389d7a53ed1eb/chr_7.zip
wget https://imputationserver.sph.umich.edu/share/results/13760c1d703c26649031064140a645de/chr_8.zip
wget https://imputationserver.sph.umich.edu/share/results/1d051ae4b068ac995440a0b66b07d147/chr_9.zip

#================================================================================ 

# paste code above into bash script called download_data
/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name ${ancestry}_impute_download \
    --script_prefix ${ancestry}_impute_download \
    --mem 6.8 \
    --nslots 4 \
    --priority 0 \
    --program bash download_data_${ancestry}

In [ ]:
# inflate chr results
for file in *zip;do
    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${ancestry}_unzip \
        --script_prefix ${ancestry}_unzip \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program unzip -P $passW $file
done

#rm -rf *zip

## EA
The zip files from Michigan Imputation Server (MIS) need to be inflated before you can begin working with them. They require a passcode that is sent by MIS to email.
* password: y3OdCfNQFzM5vJ

Then, inflate imputation results.

In [ ]:
ancestry=ea
passW="'y3OdCfNQFzM5vJ'"
cd /shared/jmarks/heroin/uhs4/genotype/imputed/004/final/$ancestry

## dowload_data_$ancestry
#================================================================================ 
# QC-results
wget https://imputationserver.sph.umich.edu/share/results/dd14ad588bed1774e46dd214e75f6e70/qcreport.html

# SNP Statistics
wget https://imputationserver.sph.umich.edu/share/results/717222dab28aec97154deb33e5c95a8c/statistics.txt

# Logs
wget https://imputationserver.sph.umich.edu/share/results/2a615aafd24e88b3b68cba4099969695/chr_1.log
wget https://imputationserver.sph.umich.edu/share/results/e7327cc29540b026099cee457b33db50/chr_10.log
wget https://imputationserver.sph.umich.edu/share/results/4f44efd9e94a624a98dfb708adb53fda/chr_11.log
wget https://imputationserver.sph.umich.edu/share/results/4600ec581447522352dfdd9620a49f40/chr_12.log
wget https://imputationserver.sph.umich.edu/share/results/5762b1538b8806c514e661326bffa2a4/chr_13.log
wget https://imputationserver.sph.umich.edu/share/results/6215bfad0b166a52c2a32af1adc3df0b/chr_14.log
wget https://imputationserver.sph.umich.edu/share/results/5e15300d16b08215e2c74f94ee193bde/chr_15.log
wget https://imputationserver.sph.umich.edu/share/results/61ff293e162800d4c16f1558651f88ea/chr_16.log
wget https://imputationserver.sph.umich.edu/share/results/2c9e78c601c4ca9b67cec404986dbbb8/chr_17.log
wget https://imputationserver.sph.umich.edu/share/results/9ec2fd857d5b28fd8217e72d1968edbc/chr_18.log
wget https://imputationserver.sph.umich.edu/share/results/7b8c14b9740500fd856fbd27f3f99a07/chr_19.log
wget https://imputationserver.sph.umich.edu/share/results/7ea880879afa12710f3f89ebc4c264a1/chr_2.log
wget https://imputationserver.sph.umich.edu/share/results/37f6ccacbd4ae279b75029cc92b709ec/chr_20.log
wget https://imputationserver.sph.umich.edu/share/results/505f88ad7583b9f27c78651c7c03cc3e/chr_21.log
wget https://imputationserver.sph.umich.edu/share/results/baf8727796cb866deb2f060c83d7b455/chr_22.log
wget https://imputationserver.sph.umich.edu/share/results/1f180f81b46f1c0d9b471b8e06ceba75/chr_3.log
wget https://imputationserver.sph.umich.edu/share/results/e8c8c5a9df9b5b8fadf83b3f947401c7/chr_4.log
wget https://imputationserver.sph.umich.edu/share/results/e5a3f4c92967eea58d85e2a798083028/chr_5.log
wget https://imputationserver.sph.umich.edu/share/results/c9e208b3bb6b968c7e2fb4afb488ee9/chr_6.log
wget https://imputationserver.sph.umich.edu/share/results/ae41fd413994d3f5c38136bd24b24759/chr_7.log
wget https://imputationserver.sph.umich.edu/share/results/c40bfcde0854e6aae28f052545118de0/chr_8.log
wget https://imputationserver.sph.umich.edu/share/results/8b6bb2e196c4c885227117d846458682/chr_9.log

# Imputation Results
wget https://imputationserver.sph.umich.edu/share/results/9d4b28150080437bab55060ab6e9b986/chr_1.zip
wget https://imputationserver.sph.umich.edu/share/results/11e3f84815b78daa835b1c1c4880e1e6/chr_10.zip
wget https://imputationserver.sph.umich.edu/share/results/77962cdd7625182b6836899148c8d17e/chr_11.zip
wget https://imputationserver.sph.umich.edu/share/results/bd9395cd24b5ad5e7556e6ce911386f8/chr_12.zip
wget https://imputationserver.sph.umich.edu/share/results/405ad34c0b04a609f393d3f89216f4a9/chr_13.zip
wget https://imputationserver.sph.umich.edu/share/results/bdb17c80d2d3d335abef0f836dd540f6/chr_14.zip
wget https://imputationserver.sph.umich.edu/share/results/efdc2db00f999e08cbfa3e7b2d83871e/chr_15.zip
wget https://imputationserver.sph.umich.edu/share/results/2eef2da367a7adc880a9dfd6381d56ee/chr_16.zip
wget https://imputationserver.sph.umich.edu/share/results/ac72539e571c930de58979848cffae1f/chr_17.zip
wget https://imputationserver.sph.umich.edu/share/results/956f12e67d6888eee1c0cbdf04b0657/chr_18.zip
wget https://imputationserver.sph.umich.edu/share/results/139f088f1990015acfcb89c8b85fbf7d/chr_19.zip
wget https://imputationserver.sph.umich.edu/share/results/47855eda544bf25a3691ea5fb6fd02e0/chr_2.zip
wget https://imputationserver.sph.umich.edu/share/results/95725a25c3902ca0e6a5bb000b824b5d/chr_20.zip
wget https://imputationserver.sph.umich.edu/share/results/be55570cba8e4652671eb546c82a0aa0/chr_21.zip
wget https://imputationserver.sph.umich.edu/share/results/586240dd2b5c37de46017a3b70206d10/chr_22.zip
wget https://imputationserver.sph.umich.edu/share/results/f08eb3936f3e6a2481dc0dd0082fec4a/chr_3.zip
wget https://imputationserver.sph.umich.edu/share/results/fa14fb6c0dbea442314bd0b9ddfa557a/chr_4.zip
wget https://imputationserver.sph.umich.edu/share/results/b84d76d7438af690c9d66530021f6d2e/chr_5.zip
wget https://imputationserver.sph.umich.edu/share/results/702f629464e4dedfeb93991085b77acd/chr_6.zip
wget https://imputationserver.sph.umich.edu/share/results/5e01222c5d26822cf4dd6c7e6d2f7755/chr_7.zip
wget https://imputationserver.sph.umich.edu/share/results/361f47e4cc291db37bfbc6fda2911187/chr_8.zip
wget https://imputationserver.sph.umich.edu/share/results/571776434d6278d92e3125530e6978c/chr_9.zip

#================================================================================ 

# paste code above into bash script called download_data
/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name ${ancestry}_impute_download \
    --script_prefix ${ancestry}_impute_download \
    --mem 6.8 \
    --nslots 4 \
    --priority 0 \
    --program bash download_data_${ancestry}

In [ ]:
# inflate chr results
for file in *zip;do
    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${ancestry}_unzip \
        --script_prefix ${ancestry}_unzip \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program unzip -P $passW $file
done

#rm -rf *zip

# Upload to S3

In [ ]:
cd /shared/jmarks/heroin/uhs4/genotype/imputed/004

for ancestry in {aa,ea};do
    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${ancestry}_upload \
        --script_prefix ${ancestry}_upload \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program aws s3 sync ${ancestry} s3://rti-heroin/uhs4/data/genotype/imputed/${ancestry}/
done